In [143]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
import requests
from bs4 import BeautifulSoup
import csv

In [144]:
place_code = {"東京":{"prec_no":"44", "block_no":"47662", "type":"s1"}, "大分":{"prec_no":"83", "block_no":"0808", "type":"a1"}, "弘前":{"prec_no":"31", "block_no":"0166", "type":"a1"}}
base_url = "http://www.data.jma.go.jp/obd/stats/etrn/view/daily_%s.php?prec_no=%s&block_no=%s&year=%s&month=%s&day=&view=p1"

# base_url = "http://www.data.jma.go.jp/obd/stats/etrn/view/daily_a1.php?prec_no=83&block_no=0933&year=2011&month=12&day=&view=p1"



In [145]:
#取ったデータをfloat型に変えるやつ。(データが取れなかったとき気象庁は"/"を埋め込んでいるから0に変える)
def str2float(str):
    try:
        return float(str)
    except:
        return 0.0

In [146]:
l = []
#都市を網羅します
for place in place_code.keys():
    #最終的にデータを集めるリスト (下に書いてある初期値は一行目。つまり、ヘッダー。)
    print(place)
    # index = place_name.index(place)
    prec_no = place_code[place]["prec_no"]
    block_no = place_code[place]["block_no"]
    data_type = place_code[place]["type"]
    
    # for文で2007年~2017年までの11回。
    for year in range(1991, 2021):
        print(year)
        # その年の1月~12月の12回を網羅する。
        for month in range(1,13):
            #print(month)
            #2つの都市コードと年と月を当てはめる。
            r = requests.get(base_url%(data_type, prec_no, block_no, year, month))
            r.encoding = r.apparent_encoding

            # まずはサイトごとスクレイピング
            soup = BeautifulSoup(r.text)
            # findAllで条件に一致するものをすべて抜き出します。
            # 今回の条件はtrタグでclassがmtxになってるものです。
            rows = soup.findAll('tr',class_='mtx')

            # 表の最初の1~4行目はカラム情報なのでスライスする。(indexだから初めは0だよ)
            # 【追記】2020/3/11 申し訳ございません。間違えてました。
            if data_type == "a1":
                rows_data = rows[3:]

                # 1日〜最終日までの１行を網羅し、取得します。
                for row in rows_data:
                    # 今度はtrのなかのtdをすべて抜き出します
                    data = row.findAll('td')

                    #１行の中には様々なデータがあるので全部取り出す。
                    # ★ポイント
                    day = data[0].findAll('a')[0].string # 日にち
                    date = str(year) + "/" + str(month) + "/" + day
                    precip_total  = data[1].string #降水量(mm)合計
                    temp_ave = data[4].string #平均気温
                    temp_high = data[5].string #最高気温
                    temp_low = data[6].string #最低気温
                    daylight = data[13].string #日照時間
                    d = {"date":date, "precip_total":precip_total, "temp_ave":temp_ave, "temp_high":temp_high, "temp_low":temp_low, "daylight":daylight, "place":place}
                    l.append(d)
            else:
                rows_data = rows[4:]
                for row in rows_data:
                    # 今度はtrのなかのtdをすべて抜き出します
                    data = row.findAll('td')

                    #１行の中には様々なデータがあるので全部取り出す。
                    # ★ポイント
                    day = data[0].findAll('a')[0].string
                    date = str(year) + "/" + str(month) + "/" + day
                    precip_total  = data[3].string #降水量(mm)合計
                    temp_ave = data[6].string #平均気温
                    temp_high = data[7].string #最高気温
                    temp_low = data[8].string #最低気温
                    daylight = data[16].string #日照時間
                    d = {"date":date, "precip_total":precip_total, "temp_ave":temp_ave, "temp_high":temp_high, "temp_low":temp_low, "daylight":daylight, "place":place}
                    l.append(d)
                    
df = pd.DataFrame(l)

東京
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
大分
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
弘前
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020


In [0]:
# Write recipe outputs
tokyo_kyushu_temp = dataiku.Dataset("tokyo_kyushu_temp")
tokyo_kyushu_temp.write_with_schema(df)